In [ ]:
# import pandas as pd
# from pathlib import Path
import requests

In [ ]:
UR_endpoint = r"https://api.beta.ons.gov.uk/v1/datasets?is_based_on=UR"

response = requests.get(UR_endpoint + "&limit=100", timeout=5) # there's 29 datasets/topic summaries for the usual residence population
UR = response.json()
dataset_id_ls = []
for item_ls in  UR["items"]:
    print(item_ls["id"])
    dataset_id_ls.append(item_ls["id"])

In [ ]:
edition = "2021"
for dataset_arg in dataset_id_ls:
    # get latest version id, using the edition endpoint
    root_dataset_url = r"https://api.beta.ons.gov.uk/v1/datasets/"

    edition_request_url = root_dataset_url + dataset_arg + "/editions/" + edition
    
    response = requests.get(edition_request_url, timeout=5) # could add a limit in here if i wanted "&limit=100"
    edition_endpoint_metadata = response.json()
    latest_version = edition_endpoint_metadata["links"]["latest_version"]["id"]

    # once you have version make the request for the link
    csv_link_request_url = edition_request_url + "/versions/" + latest_version

    response = requests.get(csv_link_request_url, timeout=5)
    dataset_metadata = response.json()
    csv_link = dataset_metadata["downloads"]["csv"]["href"]
    csvw_link = dataset_metadata["downloads"]["csvw"]["href"]

    # get csv and csvw data
    response = requests.get(csv_link, timeout=5)
    csv_bytes = response.content # content gives the content of the response in bytes
    csv_file = open(dataset_arg + "_" + edition +".csv","wb")
    csv_file.write(csv_bytes)
    csv_file.close()
    print(csv_file.name)

    response = requests.get(csvw_link, timeout=5)
    csvw_bytes = response.content # content gives the content of the response in bytes
    csvw_file = open(dataset_arg + "_" + edition +"_csvw-metadata.json","wb")
    csvw_file.write(csvw_bytes)
    csvw_file.close()
    print(csvw_file.name)

    